In [76]:
# !pip install langchain_community
# !pip install faiss-gpu
# !pip install rank_bm25 
# !pip install ragas
# !pip install datasets
# !pip install pdfplumber

In [77]:
import os
os.getcwd()

'/kaggle/working'

In [78]:
import pdfplumber

# Open the PDF file
with pdfplumber.open('/kaggle/input/star-wars/Star Wars - Brotherhood Mike Chen.pdf') as pdf:
    # Open the text file for writing
    with open('knowledge_base.txt', 'w', encoding='utf-8') as output:
        # Iterate over pages 10 to 349 (0-indexed, so subtract 1)
        for i in range(9, 349):  # Page 10 is index 9
            page = pdf.pages[i]
            text = page.extract_text() or ""  # Handle cases where text is None
            output.write(text + '\n')  # Write text to file with a newline


In [79]:
from langchain_community.document_loaders import PyMuPDFLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.schema.document import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import TextLoader

from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers import pipeline
from rank_bm25 import BM25Okapi
import numpy as np
import warnings

from datasets import Dataset
warnings.filterwarnings("ignore")
import textwrap

In [127]:
# 1. Load your text files
file_paths = ["/kaggle/working/knowledge_base.txt"]
documents = []

for file_path in file_paths:
    loader = TextLoader(file_path)
    docs = loader.load()
    documents.extend(docs)

# 2. Define chunking parameters
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,        # You can try 100, 250, 512, etc.
    chunk_overlap=512       # Try 0, 50, 100, etc.
)

# 3. Split the documents
chunks = text_splitter.split_documents(documents)


# 4. Output result
print(f"Total chunks: {len(chunks)}")
print(f"First chunk content:\n{chunks[0].page_content}")

# Optional: Save the chunks to a file
with open("chunked_output.txt", "w", encoding='utf-8') as f:
    for i, chunk in enumerate(chunks):
        f.write(f"--- Chunk {i + 1} ---\n")
        f.write(chunk.page_content + "\n\n")


Total chunks: 1198
First chunk content:
A long time ago in a galaxy far, far away….
The CLONE WARS have erupted. Caught off guard by the quickly
expanding conflict, the overwhelmed Jedi Order has rushed the
advancement of Padawans to better integrate into the Grand Army of the
Republic and assist the war effort.
Newly promoted Jedi Knight Anakin Skywalker is increasingly torn
between his growing duties to the Republic and his secret marriage to
Senator Padmé Amidala of Naboo. With his Knighting, his mentor Obi-
Wan Kenobi has been elevated to the Jedi Council under the rank of Jedi
Master.
As dark forces push the Jedi further toward their transformation from
guardians to soldiers, Anakin and Obi-Wan find themselves on equal
footing yet opposing paths, each pondering the meaning of peace and
justice during a time of war…
CHAPTER 1
RUUG QUARNOM
Cato Neimoidia was a world of mist.
High above that mist, cliffs and branches poked through, carved at all
angles into immense mountainous spires

In [128]:
# Prepare documents and their metadata
texts = [chunk.page_content for chunk in chunks]
metadata = [chunk.metadata for chunk in chunks]
print(len(texts))


1198


In [82]:
# Initialize embedding model
embedding_model = HuggingFaceEmbeddings(model_name="BAAI/bge-small-en")

# Create FAISS vector database
vectordb = FAISS.from_documents(chunks, embedding_model)

# Save FAISS index to disk for later use
vectordb.save_local("faiss_index")

# Check the number of stored documents
print(f"Number of documents in the vector store: {vectordb.index.ntotal}")

Number of documents in the vector store: 1198


In [83]:
# BM25 Indexing
tokenized_texts = [text.split() for text in texts]
bm25 = BM25Okapi(tokenized_texts)

def reciprocal_rank_fusion(results_bm25, results_embedding, k=2):
    scores = {}

    # Use document content or metadata as the key
    for rank, (doc, score) in enumerate(results_bm25):
        doc_id = doc.page_content  # Or use doc.metadata.get("source", "unknown") if available
        scores[doc_id] = scores.get(doc_id, 0) + 1 / (rank+1) # (k + rank + 1)
        print("BM25", scores[doc_id])

    for rank, (doc, score) in enumerate(results_embedding):
        doc_id = doc.page_content  # Use the same identifier
        scores[doc_id] = scores.get(doc_id, 0) + 1 / (rank+1) # (k + rank + 1)
        print("Dense", scores[doc_id])

    return sorted(scores.items(), key=lambda x: x[1], reverse=True)


# Extract page content and metadata properly
def format_response(doc):
    return f"Page {doc.metadata.get('page', 'Unknown')}: {doc.page_content.strip()}"

In [84]:
# Retrieve function
def retrieve(query, k=3):
    query_embedding = embedding_model.embed_query(query)
    results_embedding = vectordb.similarity_search_with_score_by_vector(query_embedding, k=k)
    results_embedding = sorted(results_embedding, key=lambda x: x[1], reverse=True)
    
    print("============Dense Embeddings=============")
    for doc, score in results_embedding:
        print(f"page {doc.metadata.get('page','Unknown')} - Score: {score:.4f} - {doc.page_content[:100]}...")

    # Get BM25 scores for all documents and sort to get top-k results
    results_bm25 = [(idx, bm25.get_scores(query.split())[idx]) for idx in range(len(texts))]
    results_bm25 = sorted(results_bm25, key=lambda x: x[1], reverse=True)[:k]  # Keep only top-k results
    # Convert BM25 results to (Document, score) format
    results_bm25_docs = [(Document(page_content=texts[idx], metadata=metadata[idx]), score) for idx, score in results_bm25]
   
    print("************BM25 Results*************")
    for doc, score in results_bm25_docs:
        print(f"page {doc.metadata.get('page','Unknown')} - Score: {score:.4f} - {doc.page_content[:100]}...")
    
    # Create a lookup dictionary {document content -> Document object}
    doc_lookup = {doc.page_content: doc for doc, _ in results_bm25_docs}
    doc_lookup.update({doc.page_content: doc for doc, _ in results_embedding})

    # Fuse results
    fused_results = reciprocal_rank_fusion(results_bm25_docs, results_embedding)
    
    # Format results, ensuring document IDs are mapped back to actual Documents
    return [format_response(doc_lookup[doc_id]) for doc_id, _ in fused_results if doc_id in doc_lookup]

    #fused_results = reciprocal_rank_fusion(results_bm25, results_embedding)
    #return [(texts[idx], metadata[idx]["page"] if "page" in metadata[idx] else "Unknown") for idx, _ in fused_results]


In [105]:
from langchain.schema import Document

def retrieve_dense(query, k=3):
    query_embedding = embedding_model.embed_query(query)
    results_embedding = vectordb.similarity_search_with_score_by_vector(query_embedding, k=k)
    
    # Optionally sort descending by score if needed
    results_embedding = sorted(results_embedding, key=lambda x: x[1], reverse=True)
    
    print("============Dense Embeddings=============")
    for doc, score in results_embedding:
        print(f"page {doc.metadata.get('page','Unknown')} - Score: {score:.4f} - {doc.page_content[:100]}...")
    
    # Return just the documents (or both doc and score if you want)
    return [format_response(doc) for doc, _ in results_embedding]


In [144]:
# Query example
question = "What does Anakin Skywalker gifts Padmé Amidala during the night at the Uhmandasee Market and Please Explain it's design?"
retrieved_responses = retrieve_dense(question, k=5)

============Dense Embeddings=============
page Unknown - Score: 0.2330 - Padmé held it up, the harsh light from the vendor’s window enough to
put the object in full view. “I...
page Unknown - Score: 0.2329 - industrial fumes puffing from nearby stacks and vents were fog.
The artificial night sky made the un...
page Unknown - Score: 0.2282 - deeper. He paused, letting Anakin take a sip of soup before moving on to a
new topic, something just...
page Unknown - Score: 0.2188 - as stools and toward the path leading out of Uhmandasee Market.
“Wait, we’re not done with the sunda...
page Unknown - Score: 0.2150 - sight of her walking into Watto’s shop that he blurted out the first thing that
came to mind. “I hav...


In [86]:
# # Query processing
# question = "What was the cause of the bombing on Cato Neimoidia in Brotherhood?"
# retriever = vectordb.as_retriever(search_kwargs={"k": 10})
# docs = retriever.get_relevant_documents(question)

# # Print results
# for i, doc in enumerate(docs, 1):
#     page_number = doc.metadata.get('page', 'Unknown')
#     # print(f"Document {i} - Page {page_number} - Score: {doc.metadata.get('score', 'N/A')}")
#     print(doc.page_content[:])  # Print first 500 characters of each result
#     print("-" * 80)

In [110]:
# model_name = "tiiuae/Falcon3-3B-Instruct"
model_name = "Qwen/Qwen2.5-7B-Instruct"
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto", #device_map='cuda'
    torch_dtype="auto",
    trust_remote_code=True,
)
tokenizer = AutoTokenizer.from_pretrained(model_name)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [111]:
# Create a pipeline
generator = pipeline(
"text-generation",
model=model,
tokenizer=tokenizer,
return_full_text=False,
max_new_tokens=5000,
do_sample=False
)

Device set to use cuda:0


In [89]:
# # Extract page content and metadata properly
# def format_response(doc):
#     return f"Page {doc.metadata.get('page', 'Unknown')}: {doc.page_content.strip()}"

# # Handle cases where fewer than 3 results are returned
# retrieved_responses = [format_response(doc) for doc in docs[:3]]
# while len(retrieved_responses) < 3:
#     retrieved_responses.append("No relevant information.")  # Fill missing slots

In [145]:
# Construct the RAG prompt
prompt = f"""
You are an AI assistant tasked with answering questions based on retrieved knowledge.

### **Retrieved Information**:
1. {retrieved_responses[0]}
2. {retrieved_responses[1]}
3. {retrieved_responses[2]}
4. {retrieved_responses[3]}
5. {retrieved_responses[4]}




### **Question**:
{question}

### **Instructions**:
- Integrate the key points from all retrieved responses into a **cohesive, well-structured answer**.
- If the responses are **contradictory**, mention the different perspectives.
- If none of the retrieved responses contain relevant information, reply:
  **"I couldn't find a good response to your query in the database."**
"""



In [146]:
for i in range(0,len(retrieved_responses)):
    print(retrieved_responses[i])
    print("-------")

Page Unknown: Padmé held it up, the harsh light from the vendor’s window enough to
put the object in full view. “Is this…?”
“My Padawan braid.” She traced the braid, now coiled into a spiral and
encased in carbonite as it sat within the pendant’s simple design. “To new
beginnings.”
Even in the dim light, Anakin could see the flush come to Padmé’s
cheeks, and though she’d been known to sway entire systems through the
power of words, her mouth remained open yet silent. But she didn’t need to
say anything.
The look in her eyes was enough. She leaned against him, and even
though she put the chain around her neck, her hands still held on to the
pendant, as if it could keep them together despite the galactic turmoil
constantly around them. “Okay,” he said after a minute. She turned to him,
inquisitively. “Now I’m going to try the franikhad.”
—
It was close to Tatooine cooking—close enough that Anakin considered
ways he could discreetly get credits, find a speeder, and enjoy it from time
----

In [147]:
# Use Qwen2.5 3B with the correct message format
messages = [
    {"role": "user", "content": prompt}
]

# Generate output using the model
output = generator(messages)

# Print formatted response
print(textwrap.fill(output[0]["generated_text"], width=80))

During the night at the Uhmandasee Market, Anakin Skywalker gifts Padmé Amidala
a piece of his Padawan braid encased in carbonite and set within a simple
pendant. This gift symbolizes their commitment to each other amidst the turmoil
of the galaxy.  The pendant features a disk-shaped gold pendant with a necklace
chain. The key element of the gift is the braid itself, which has been coiled
into a spiral and encased in carbonite. This design choice is significant
because:  1. **Personal Connection**: The braid represents a piece of Anakin,
symbolizing his dedication and love for Padmé. 2. **Carbonite Encasement**: The
use of carbonite adds a layer of permanence and preservation to the gift,
suggesting that their bond is strong and enduring. 3. **Simple Design**: Despite
the personal significance of the braid, the overall design of the pendant is
described as "simple," indicating that the true value lies in the sentiment
behind the gift rather than its appearance.  This gift serves as a t